<a href="https://colab.research.google.com/github/rupaidutta66/Cmputer_Vison/blob/main/Vsion_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from google.colab import drive
drive.mount('/content/drive')
import cv2
import easyocr
import os
import re

Mounted at /content/drive


In [5]:


input_file = '/content/drive/MyDrive/Train_coach_number.mkv'
subprocess.run(['ffmpeg',
                '-i',
                input_file,
                '-qscale',
                '0',
                'Train_coach_number.mp4',
                '-loglevel',
                'quiet']
              )



input_video_path = '/content/Train_coach_number.mp4'
#output_images_folder = 'frames_images'
output_data_file = 'extracted_vehicle_numbers.txt'


#os.makedirs(output_images_folder, exist_ok=True)
#os.makedirs(output_data_file, exist_ok=True)


cap = cv2.VideoCapture(input_video_path)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

reader = easyocr.Reader(['en'])

frame_text_dict = {}
previous_text = ""

for frame_num in range(n_frames):
    ret, img = cap.read()
    if not ret:
        break

    bounds = reader.readtext(np.array(img), min_size=0, slope_ths=0.2, ycenter_ths=0.6, width_ths=0.7, decoder='beamsearch', beamWidth=10)


    frame_text = ""


    for text_info in bounds:
        text = text_info[1]
        match = re.search(r'\b\d{6}/\w+\b', text)
        if match:
            frame_text = match.group()
            break

    # Save the frame image
    '''frame_image_filename = os.path.join(output_images_folder, f'frame_{frame_num}.png')
    cv2.imwrite(frame_image_filename, img)
    '''

    if frame_text and frame_text != previous_text:

        if len(frame_text) >= 7:
            frame_text_dict[frame_num] = frame_text
            previous_text = frame_text
    elif frame_text and frame_text == previous_text:
        continue

cap.release()

with open(output_data_file, 'w') as file:
    for frame_num, text in frame_text_dict.items():
        file.write(f"Frame {frame_num}: {text}\n")


In [9]:

frames_to_exclude = [212, 222, 339, 372, 422, 588, 589, 649]

input_data_file = '/content/extracted_vehicle_numbers.txt'

with open(input_data_file, 'r') as file:
    lines = file.readlines()

lines_to_keep = []

for line in lines:
    frame_num, text = re.match(r'Frame (\d+): (.+)', line).groups()

    if int(frame_num) not in frames_to_exclude:
        lines_to_keep.append(line)

with open(input_data_file, 'w') as file:
    file.writelines(lines_to_keep)
